In [12]:
%matplotlib inline
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from nltk.stem.wordnet import WordNetLemmatizer
import re
from sklearn.linear_model import LogisticRegression as LR
import sklearn.metrics as me

In [2]:
import sys
a = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = a

# 1.Sentiment Analysis

#### Load data

In [3]:
dt = np.dtype([('text','object'), ('label', 'int16')])

amazon = np.loadtxt("data/amazon_cells_labelled.txt", dtype=dt, delimiter='\t', comments="THISISCOMMENT")
yelp = np.loadtxt("data/yelp_labelled.txt", dtype=dt, delimiter='\t', comments="THISISCOMMENT")
imdb = np.loadtxt("data/imdb_labelled.txt", dtype=dt, delimiter='\t', comments="THISISCOMMENT")

train_list = []
test_list = []
test_ratio = 0.2

for d in [amazon, yelp, imdb]:
    tr, te = train_test_split(d, test_size=test_ratio, stratify=d['label'])
    train_list.append(tr)
    test_list.append(te)

train = np.row_stack((x.reshape(-1, 1) for x in train_list))
test = np.row_stack((x.reshape(-1, 1) for x in test_list))

pos_ratio = (train[train['label'] == 1].shape[0] + test[test['label'] == 1].shape[0]) / float(train.shape[0] + test.shape[0])
print "Positive label ratio is %f" % pos_ratio

Positive label ratio is 0.500000


In [229]:
stopwords.words('english')

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

In [18]:
import string
class BagOfWordsVectorizer(object):
    def __init__(self):
        self.func_lemmatize = WordNetLemmatizer().lemmatize

        self.STRIP_CHARS = string.punctuation + ' '
        self.PUNCTUATIONS = set(string.punctuation)

#         self.STOP_WORDS = set(stopwords.words('english'))
        self.STOP_WORDS = set([])


        self.APOSTROPHE_DICT = {"don't": "donot", "doesn't": "donot", "didn't": "donot", 
                                "isn't": "isnot", "aren't": "isnot", 
                                "i'm": "i", "you're": "you", "they're": "they", "he's": "he", "she's": "she", 
                                "it's": "it", "i've": "i", "you've": "you",
                                "ha": "has"}
        self._vocabulary = None
        self._lookup_voc = None
        
    def _strip_and_lemmatize(self, word, chars):
#         word = str(self.func_lemmatize(word.strip(chars)))
        word = str(self.func_lemmatize(word))
        
        if word in self.APOSTROPHE_DICT:
            word = self.APOSTROPHE_DICT[word]
            
#         return word
        return word.translate(None, self.STRIP_CHARS)
    
    def _tokenize_and_clean(self, sentence):
#         _ = [self._strip_and_lemmatize(s, self.STRIP_CHARS) 
#              for s in sentence.lower().split(' ') if (s not in self.STOP_WORDS and len(s) > 0)]
        _ = [self._strip_and_lemmatize(s, self.STRIP_CHARS) 
             for s in re.split(r'[^a-z\']', sentence.lower()) if (s not in self.STOP_WORDS and len(s) > 0)]
    
        return filter(lambda s: len(s) > 0 and 
                               s not in self.STOP_WORDS and 
                               s not in self.PUNCTUATIONS, _)
    def tokenize(self, X):
        return map(self._tokenize_and_clean, X)
    
    
    def _l1(self, x):
        _l = len(x.nonzero()[0])
        if _l > 0:
            return x / float(_l)
        else:
            return np.zeros((len(x),))
        
    def _l2(self, x):
        _l = len(x.nonzero()[0])
        if _l > 0:
            return x / float(np.sqrt(np.dot(x, x.T)))
        else:
            return np.zeros((len(x),))

    def l1_normalize(self, X):
        return np.apply_along_axis(lambda x: self._l1(x), 1, X)

    def l2_normalize(self, X):
        return np.apply_along_axis(lambda x: self._l2(x), 1, X)
    
    
    def _bag_of_words(self, words):
        bag = np.zeros((self._voc_len,), dtype=np.int)
        for word in words:
            try:
                bag[self._vocabulary[word]] += 1
            except KeyError:
                continue
        return bag
    
        
    def fit(self, X):
        self.fit_transform(X)
        return self
    
    def fit_transform(self, X, normalize=None):
        # 1. build vocabulary
        tokens = self.tokenize(X)
        
        _ = set(np.concatenate(tokens))
        
        self._lookup_voc = {}
        self._vocabulary = {}
        for i, x in enumerate(_):
            self._lookup_voc[i] = x
            self._vocabulary[x] = i
        
        self._voc_len = len(_)
        
        # 2. generate X
        _ = np.array(map(self._bag_of_words, tokens))
        if normalize is None:
            return _
        elif normalize == 'l2':
            return self.l2_normalize(_)
        else:
            return self.l1_normalize(_)
            
    def transform(self, X, normalize=None):
        tokens = self.tokenize(X)
#         print tokens
        if self._vocabulary is not None:
            _ = np.array(map(self._bag_of_words, tokens)) 
            
            if normalize is None:
                return _
            elif normalize == 'l2':
                return self.l2_normalize(_)
            else:
                return self.l1_normalize(_)
        else:
            return None
            


In [5]:
import random
class KMeans():
    def __init__(self, max_iter=1000):
        self.max_iter = max_iter
        
    def fit(self, X, k):
        self.X = X
        self.N, self.p = X.shape
        self.k = k
        
        _ = set([])
        while len(_) < k:
            _.add(random.randint(0, self.N - 1))
        self.center = np.copy(X[tuple(_), :])
        assert self.center.shape == (self.k, self.p)
        
        self.status = np.zeros((self.N,))
        
        converge = False
        iter_count = 0
        while not converge and iter_count < self.max_iter:
            converge = True
            _status = np.apply_along_axis(self._get_center_idx, 1, X)
            
            if not np.array_equal(self.status, _status):
                converge = False
                self._update_center(_status)
                self.status = _status
            
            iter_count += 1
        self.iter_count = iter_count
        
    def _get_center_idx(self, x):
        return np.argmin(np.sum((self.center - x) ** 2, axis=1), axis=0)
        
    def _update_center(self, status):
        for i in range(self.k):
            _center = np.mean(self.X[status == i], axis=0)
            self.center[i] = _center
    
    def get_center(self):
        return self.center # shape: k*p
    
    def get_status(self):
        return self.status
    
    def get_cluster(self, k):
        return self.X[self.status == k]

In [364]:
kmeans = KMeans()
bowv = NGramVectorizer(2)
train_bag = bowv.fit_transform(train['text'][:, 0], normalize=None)
kmeans.fit(train_bag, 2)
train_center = kmeans.get_center()
train_status = kmeans.get_status()
cluster0 = train['label'][train_status == 0, 0]
cluster1 = train['label'][train_status == 1, 0]
print np.sum(cluster0)
print cluster0.shape[0] - np.sum(cluster0)
print np.sum(cluster1)
print cluster1.shape[0] - np.sum(cluster1)

0
1
1200
1199


In [340]:
cluster0 = [bowv._lookup_voc[_idx] for _idx in train_center[0].nonzero()[0]]
cluster1 = [bowv._lookup_voc[_idx] for _idx in train_center[1].nonzero()[0]]

In [346]:
cluster0

['limited',
 'todays',
 'secondly',
 'yellow',
 'sleek',
 'four',
 'sleep',
 'appetite',
 'poorly',
 'whose',
 'replacementr',
 'buddy',
 'spew',
 'rpg',
 'sweetest',
 'charles',
 'insipid',
 'whatsoever',
 'under',
 'rickman',
 'lord',
 'welcome',
 'worth',
 'sinking',
 'struck',
 'rescue',
 'shoot',
 'smack',
 'every',
 'jack',
 'bertolucci',
 'trailer',
 'believed',
 'school',
 'wooden',
 'loneliness',
 'coppola',
 'upload',
 'girolamo',
 'frozen',
 'haggis',
 'falwell',
 'ups',
 'enjoy',
 'force',
 'specially',
 'phones',
 'consistent',
 'bacon',
 'budget',
 'chef',
 'surrounding',
 'second',
 'louder',
 'mesmerising',
 'gorman',
 'oscar',
 'monster',
 'even',
 'established',
 'everyones',
 'microsofts',
 'ruthless',
 'cooking',
 'asia',
 'achille',
 'veggitarian',
 'above',
 'new',
 'net',
 'ever',
 'told',
 'simmering',
 'hero',
 'philippa',
 'reporter',
 'join',
 'never',
 'disposable',
 'meh',
 'here',
 'screamy',
 'protection',
 'greedy',
 'cardboard',
 'mozzarella',
 'dee',
 

##### logistic regression

In [19]:

for n in [None, 'l1', 'l2']:
    bowv = BagOfWordsVectorizer()
    train_bag = bowv.fit_transform(train['text'][:,0], normalize=n)
    test_bag = bowv.transform(test['text'][:,0], normalize=n)

    cls = LR()
    cls.fit(train_bag, train['label'][:,0])
    _pred = cls.predict(test_bag)
    print n, len((_pred == test['label'][:,0]).nonzero()[0]) / float(test_bag.shape[0])
    print me.confusion_matrix(test['label'][:,0], _pred)
print '---'

# for n in [None, 'l1', 'l2']:
#     bowv = NGramVectorizer(2)
#     train_bag = bowv.fit_transform(train['text'][:,0], normalize=n)
#     test_bag = bowv.transform(test['text'][:,0], normalize=n)

#     cls = LR()
#     cls.fit(train_bag, train['label'][:,0])
#     _pred = cls.predict(test_bag)
#     print n, len((_pred == test['label'][:,0]).nonzero()[0]) / float(test_bag.shape[0])
#     print me.confusion_matrix(test['label'][:,0], _pred)



None 0.836666666667
[[254  46]
 [ 52 248]]
l1 0.75
[[224  76]
 [ 74 226]]
l2 0.788333333333
[[235  65]
 [ 62 238]]
---


In [30]:
bowv = NGramVectorizer(2)
train_bag = bowv.fit_transform(train['text'][:,0], normalize=None)
test_bag = bowv.transform(test['text'][:,0], normalize=None)

cls = LR()
cls.fit(train_bag, train['label'][:,0])
coef = cls.coef_[0]
idx = np.argsort(coef)
print '-good-'
for i in idx[::-1][:20]:
    print '%s; ' % bowv._lookup_voc[i],
print '\n-bad-'
for i in idx[:20]:
    print '%s; ' % bowv._lookup_voc[i],

-good-
work great;  the best;  a great;  very good;  i love;  love this;  great phone;  all the;  is great;  you wont;  wa good;  i liked;  would recommend;  to go;  loved it;  so good;  the price;  work fine;  loved the;  better than;  
-bad-
the worst;  would not;  not good;  piece of;  waste of;  wa terrible;  very disappointed;  doe not;  donot work;  so bad;  the battery;  at all;  not work;  i not;  very bad;  wa not;  just donot;  very disappointing;  the script;  it just; 


# N-gram

In [6]:
class NGramVectorizer(BagOfWordsVectorizer):
    def __init__(self, N):
        super(NGramVectorizer, self).__init__()
        self.N = N
        
    def _gen_ngram(self, tokens):
        r = []
        for i in range(0, len(tokens) - self.N + 1):
            r.append(" ".join(tokens[i:i+self.N]))
        return r
        
    def tokenize(self, X):
        _ = super(NGramVectorizer, self).tokenize(X)
        return map(self._gen_ngram, _)

## PCA

In [7]:
from numpy.linalg import svd
class PCA(object):
    def fit_transform(self, X, q):
        self.u = np.mean(X, axis=0)
        self._X = X - self.u
        self.V = svd(self._X)[-1]
        return self._X.dot(self.V[:q, :].T)
    
    def transform(self, X, q):
        return (X - self.u).dot(self.V[:q, :].T)

In [26]:
print '-- KMeans --'
for q in [200, 500]:
    bowv = BagOfWordsVectorizer()
    pca = PCA()
    train_bag = pca.fit_transform(bowv.fit_transform(train['text'][:,0], normalize='l2'), q)
    test_bag = pca.transform(bowv.transform(test['text'][:,0], normalize='l2'), q)
    
    kmeans = KMeans()
    kmeans.fit(train_bag, 2)
    train_center = kmeans.get_center()
    train_status = kmeans.get_status()
    cluster0 = train['label'][train_status == 0, 0]
    cluster1 = train['label'][train_status == 1, 0]
    print "cluster0", np.sum(cluster0), 
    print cluster0.shape[0] - np.sum(cluster0)
    print "cluster1", np.sum(cluster1), 
    print cluster1.shape[0] - np.sum(cluster1)
    print '--'

-- KMeans --
cluster0 299 285
cluster1 901 915
--
cluster0 177 205
cluster1 1023 995
--


In [25]:
print '-- Bag-of-words --'
for q in [200, 500]:
    print '-- q = %d -- ' % q
    for n in [None, 'l1', 'l2']:
        bowv = BagOfWordsVectorizer()
        pca = PCA()
        train_bag = pca.fit_transform(bowv.fit_transform(train['text'][:,0], normalize=n), q)
        test_bag = pca.transform(bowv.transform(test['text'][:,0], normalize=n), q)

        cls = LR()
        cls.fit(train_bag, train['label'][:,0])
        _pred = cls.predict(test_bag)
        print n, len((_pred == test['label'][:,0]).nonzero()[0]) / float(test_bag.shape[0])
        print me.confusion_matrix(test['label'][:,0], _pred)

-- Bag-of-words --
-- q = 200 -- 
None 0.768333333333
[[232  68]
 [ 71 229]]
l1 0.736666666667
[[223  77]
 [ 81 219]]
l2 0.755
[[225  75]
 [ 72 228]]
-- q = 500 -- 
None 0.798333333333
[[241  59]
 [ 62 238]]
l1 0.746666666667
[[224  76]
 [ 76 224]]
l2 0.775
[[231  69]
 [ 66 234]]


In [1]:
%whos

Interactive namespace is empty.
